In [1]:
#push!(LOAD_PATH,"../src/")
#using Revise
using ProgressBars
using ITensors
using PastaQ
using RandomMeas

In [2]:
#Perform RM in the first experiment
N = 6
circuit = randomcircuit(N, depth=2)
noisemodel1 = (1 => ("depolarizing", (p = 0,)),2 => ("depolarizing", (p = 0.05,)))
ρ1 = runcircuit(circuit;noise = noisemodel1);
ξ = firstsiteinds(ρ1;plev=0)
println(ξ)
println("state prepared in the `experiment' 1 ")

Index{Int64}[(dim=2|id=101|"Qubit,Site,n=1"), (dim=2|id=444|"Qubit,Site,n=2"), (dim=2|id=432|"Qubit,Site,n=3"), (dim=2|id=708|"Qubit,Site,n=4"), (dim=2|id=859|"Qubit,Site,n=5"), (dim=2|id=77|"Qubit,Site,n=6")]
state prepared in the `experiment' 1 


In [3]:
#Perform Randomized measurements
nu=200 #Number of random unitaries
NM=1000 #Number of projective measurements per unitary
u = Vector{Vector{ITensor}}()
data1 = zeros(Int8,(nu,NM,N)) #Data storage
for r in ProgressBar(1:nu, printing_delay=2)
            push!(u,get_rotations(ξ,1)) #generate random unitary
            data1[r,:,:] = get_RandomMeas(ρ1,u[r],NM) #data acquisation in simulated quantum device
end

0.0%┣                                            ┫ 0/200 [00:04<-12:-14, -4s/it]
0.5%┣▏                                         ┫ 1/200 [00:18<Inf:Inf, InfGs/it]
100.0%┣██████████████████████████████████████████┫ 200/200 [00:18<00:00, 11it/s]
100.0%┣██████████████████████████████████████████┫ 200/200 [00:18<00:00, 11it/s]


In [4]:
#Perform RM in the second experiment
noisemodel2 = (1 => ("amplitude_damping", (γ = 0.1,)),2 => ("amplitude_damping", (γ = 0.1,)))
ρ2 = runcircuit(circuit;noise = noisemodel2);
ξ2 = [firstind(ρ2[i],tags="Site",plev=0) for i in 1:N]
for i in 1:N
    replaceind!(ρ2[i],ξ2[i],ξ[i])
    replaceind!(ρ2[i],ξ2[i]',ξ[i]')
end
println("state prepared in the `experiment' 2 ")
data2 = zeros(Int8,(nu,NM,N))
for r in ProgressBar(1:nu, printing_delay=2)
                data2[r,:,:] = get_RandomMeas(ρ2,u[r],NM)
end

┌ Warning: Noise model not defined for 2-qubit gates! Applying tensor-product noise instead.
└ @ PastaQ ~/.julia/packages/PastaQ/IVHHb/src/circuits/noise.jl:177
┌ Warning: Noise model not defined for 2-qubit gates! Applying tensor-product noise instead.
└ @ PastaQ ~/.julia/packages/PastaQ/IVHHb/src/circuits/noise.jl:177
┌ Warning: Noise model not defined for 2-qubit gates! Applying tensor-product noise instead.
└ @ PastaQ ~/.julia/packages/PastaQ/IVHHb/src/circuits/noise.jl:177
┌ Warning: Noise model not defined for 2-qubit gates! Applying tensor-product noise instead.
└ @ PastaQ ~/.julia/packages/PastaQ/IVHHb/src/circuits/noise.jl:177
┌ Warning: Noise model not defined for 2-qubit gates! Applying tensor-product noise instead.
└ @ PastaQ ~/.julia/packages/PastaQ/IVHHb/src/circuits/noise.jl:177


state prepared in the `experiment' 2 


0.0%┣                                              ┫ 0/200 [00:00<00:00, -0s/it]
100.0%┣█████████████████████████████████████████┫ 200/200 [00:01<00:00, 362it/s]
100.0%┣█████████████████████████████████████████┫ 200/200 [00:01<00:00, 362it/s]


In [5]:
#Build histograms for each unitary
P1 = Vector{ITensor}()
P2 = Vector{ITensor}()
for r in 1:nu
                push!(P1,get_Born(data1[r,:,:],ξ))
                push!(P2,get_Born(data2[r,:,:],ξ))
end

In [6]:
#Uncomment if you want to see an animation
# using Plots
# anim = @animate for r in 1:nu
#     y1 =  reshape(array(P1[r]),(2^N))
#     y2 =  reshape(array(P2[r]),(2^N))
#     bar(y1,label="Experiment 1")
#     bar!(y2,label="Experiment 2")
#     xlabel!("bitstring index")
#     ylabel!("Probabilities")
#     title!("Random unitary $(r)")
# end
# gif(anim,fps=0.5)

In [7]:
overlaps = 0
for r in 1:nu
    global overlaps += get_overlap(P1[r],P2[r],ξ,N)/nu
end
purity1s = get_purity_hamming(data1,ξ)
purity2s = get_purity_hamming(data2,ξ)
Fs = overlaps/max(purity1s,purity2s)
println("Estimated overlap ",overlaps)
println("Estimated purity1 ",purity1s)
println("Estimated purity2 ",purity2s)
println("Estimated fidelity ",Fs)

Estimated overlap 0.5597329750000005
Estimated purity1 0.6904987687687688
Estimated purity2 0.469887897897898
Estimated fidelity 0.8106212499090515


In [8]:
overlap = real(inner(ρ1,ρ2))
purity1 = get_purity(ρ1)
purity2 = get_purity(ρ2)
F = overlap/max(purity1,purity2)
println("overlap ",overlap)
println("purity1 ",purity1)
println("purity2 ",purity2)
println("fidelity ",F)

overlap 0.5285036883241412
purity1 0.653727066923766
purity2 0.44761686500633346
fidelity 0.8084470034430628
